<br>
<center><h1>Nutika otsingu liidestamine<br> Riigi Teataja</br> infosüsteemiga</h1></center>
<br>
<center><h3>Sven Laur* & Tarmo Vaino</h3> </center>


# Status quo

<img src="illustrations/current_architecture.png">

In [1]:
from predefined_functions import search_rt_caption

search_rt_caption('presidendi')

,document_type,document_title,commencement_date,global_id
0,seadus,Riigikogu ja Vabariigi Presidendi poolt nimeta...,2014-01-01,129122012010
1,seadus,Riigikogu ja Vabariigi Presidendi poolt nimeta...,2013-01-01,128122011068
2,seadus,Riigikogu ja Vabariigi Presidendi poolt nimeta...,2012-01-01,13262582
3,seadus,Vabariigi Presidendi ametihüve seadus,2024-01-01,106072023113
4,seadus,Vabariigi Presidendi ametihüve seadus,2018-01-01,122062016013
5,seadus,Vabariigi Presidendi ametihüve seadus,2016-07-01,113122014059
6,seadus,Vabariigi Presidendi ametihüve seadus,2014-01-01,129122011152
7,seadus,Vabariigi Presidendi ametihüve seadus,2012-01-01,128122010026
8,seadus,Vabariigi Presidendi ja Vabariigi Valitsuse li...,2004-07-01,769953
9,seadus,Vabariigi Presidendi töökorra seadus,2017-07-07,127062017011


# Nutika otsingu võimalikud liidestuspunktid

<img src="illustrations/architecture.png">

# Veebilehe poolt väljakutsutavad veebiteenused
<u>Universiaalsed veebiteenustel põhinev lahendus</u>

* [analyser](https://github.com/estnltk/smart-search/tree/main/api/api_analyser) -- tekstide morfoloogiline analüsaator 
* [generator](https://github.com/estnltk/smart-search/tree/main/api/api_generator) -- sõnavormide generaator
* [speller](https://github.com/estnltk/smart-search/tree/main/api/api_speller) -- kirjavigade parandaja

Lahenduse põhimõtteline kondikava

```
wordform = spell_text(search_word)
lemma = analyze_text(wordform)
search_forms = generate_all_worldforms(lemma)
results = [fetch_documents(query_word) for query_word in search_forms]
```

In [2]:
import json
import requests

# Analüüsitav tekst
text = 'president' #'ametiraha'

ANALYZER_QUERY = "https://smart-search.tartunlp.ai/api/analyser/process"
HEADERS = {"Content-Type": "application/json; charset=utf-8"}
POST_DATA_TEMPLATE = {'params': {"vmetajson": ["--guess"]}, 'content': text}

# Päring veebiteenusele
response = requests.post(ANALYZER_QUERY, json=POST_DATA_TEMPLATE, headers=HEADERS)
assert response.ok, "Webservice failed"
response = response.json()
print(list(response.keys()))
print(f'[annotations] --> {list(response["annotations"].keys())}')

['annotations', 'content', 'params']
[annotations] --> ['tokens']


In [3]:
print(json.dumps(response['annotations']['tokens'], indent=4))

[
    {
        "features": {
            "complexity": 1,
            "mrf": [
                {
                    "ending": "0",
                    "fs": "sg n",
                    "kigi": "",
                    "komponendid": [],
                    "lemma": "president",
                    "lemma_ma": "president",
                    "lisam\u00e4rkideta": "president",
                    "pos": "S",
                    "source": "P"
                }
            ],
            "token": "president"
        }
    }
]


In [4]:
from pandas import concat
from predefined_functions import analyze_text
from predefined_functions import spell_text
from predefined_functions import generate_all_wordforms

In [ ]:
display(spell_text('presidemt'))

In [ ]:
display(analyze_text('presidendi ametiraha'))

In [11]:
tbl = generate_all_wordforms('president')
concat([tbl[:12], tbl[12:24].reset_index(drop=True), tbl[24:].reset_index(drop=True)], axis=1)

,input,wordform,input,wordform,input,wordform
0,president,presidendeks,president,presidendile,president,presidentidega
1,president,presidendel,president,presidendilt,president,presidentideks
2,president,presidendele,president,presidendina,president,presidentidel
3,president,presidendelt,president,presidendini,president,presidentidele
4,president,presidendes,president,presidendis,president,presidentidelt
5,president,presidendesse,president,presidendisse,president,presidentidena
6,president,presidendest,president,presidendist,president,presidentideni
7,president,presidendi,president,presidendita,president,presidentides
8,president,presidendid,president,president,president,presidentidesse
9,president,presidendiga,president,presidente,president,presidentidest


In [10]:
tbl = generate_all_wordforms('Srasbourgh', foreign_name=True)
concat([
    tbl[:20], 
    tbl[20:40].reset_index(drop=True), 
    tbl[40:60].reset_index(drop=True),
    tbl[60:].reset_index(drop=True)
], axis=1)

,input,wordform,input,wordform,input,wordform,input,wordform
0,Srasbourgh,Srasbourgh,Srasbourgh,Srasbourghi'l,Srasbourgh,Srasbourghi-desse,Srasbourgh,Srasbourghidele
1,Srasbourgh,Srasbourgh'e,Srasbourgh,Srasbourghi'le,Srasbourgh,Srasbourghi-dest,Srasbourgh,Srasbourghidelt
2,Srasbourgh,Srasbourgh-e,Srasbourgh,Srasbourghi'lt,Srasbourgh,Srasbourghi-deta,Srasbourgh,Srasbourghidena
3,Srasbourgh,Srasbourghe,Srasbourgh,Srasbourghi'na,Srasbourgh,Srasbourghi-ga,Srasbourgh,Srasbourghideni
4,Srasbourgh,Srasbourghi,Srasbourgh,Srasbourghi'ni,Srasbourgh,Srasbourghi-ks,Srasbourgh,Srasbourghides
5,Srasbourgh,Srasbourghi'd,Srasbourgh,Srasbourghi's,Srasbourgh,Srasbourghi-l,Srasbourgh,Srasbourghidesse
6,Srasbourgh,Srasbourghi'de,Srasbourgh,Srasbourghi'sid,Srasbourgh,Srasbourghi-le,Srasbourgh,Srasbourghidest
7,Srasbourgh,Srasbourghi'dega,Srasbourgh,Srasbourghi'sse,Srasbourgh,Srasbourghi-lt,Srasbourgh,Srasbourghideta
8,Srasbourgh,Srasbourghi'deks,Srasbourgh,Srasbourghi'st,Srasbourgh,Srasbourghi-na,Srasbourgh,Srasbourghiga
9,Srasbourgh,Srasbourghi'del,Srasbourgh,Srasbourghi'ta,Srasbourgh,Srasbourghi-ni,Srasbourgh,Srasbourghiks


In [ ]:
def smart_search_rt_caption(wordform:str, match_type:str='koik_sonad', **kwargs):

    # Correct and extend search query 
    search_words = spell_text(wordform)['suggestion']
    search_lemmas = concat([analyze_text(word)['lemma'] for word in search_words], axis=0)
    search_wordforms = concat([generate_all_wordforms(lemma)['wordform'] for lemma in search_lemmas], axis=0)

    # Interact with RT webservice
    result = [None] * len(search_wordforms)
    for i, wordform in enumerate(search_wordforms): 
        print('#', end='')
        result[i] = search_rt_caption(wordform, match_type=match_type, **kwargs)
        if i % 40 == 19:
            print(' ', end="")
        elif  i % 40 == 39:
            print()
    
    # Reshape the outcome
    return (concat(result, axis=0)
            .sort_values(['document_title', 'commencement_date'], ascending=[True, False])
            .reset_index(drop=True))  

In [ ]:
smart_search_rt_caption('president') # bresident kala 

# Veebilehe poolt väljakutsutavad veebiteenused

<u>Riigi Teataja tekstide sisu arvestav lahendus</u>

* [query_extender](https://github.com/estnltk/smart-search/tree/main/api/api_query_extender) -- laiendab otsisõna minimaalseks hulgaks RT veebiteenusele suunatud sisendiks. 

Lahenduse põhimõtteline kondikava

```
search_forms = extend_query(search_word)
results = [fetch_documents(query_word) for query_word in search_forms]
```

Eeliseks on selle kiirus. Päringut laiendatakse minimaalseks hulgaks alampäringuteks. <br/>
Probleemiks on veebiteenuse ja andmebaasi sisu omavahelises kooskõlas hoidmine.



In [ ]:
words = ['president','presidemt', 'amet'] #bresident, presidemt
ANALYSER = "https://smart-search.tartunlp.ai/api/query_extender/json"
HEADERS = {"Content-Type": "application/json; charset=utf-8"}
POST_DATA_TEMPLATE = {"params":{"otsi_liitsõnadest": 'true'}, "tss": "\t".join(words)}
response = requests.post(ANALYSER, json=POST_DATA_TEMPLATE, headers=HEADERS)
assert response.ok, "Webservice failed"
response = response.json()

print(list(response.keys()))
response['president']

In [ ]:
from predefined_functions import generate_wordforms_with_query_extender
display(generate_wordforms_with_query_extender('president', search_sublemmas=False))
display(generate_wordforms_with_query_extender('president', search_sublemmas=True))

In [ ]:
display(generate_wordforms_with_query_extender('bresident', search_sublemmas=True))
display(generate_wordforms_with_query_extender('presidemt', search_sublemmas=True))

In [ ]:
display(generate_wordforms_with_query_extender('NATO', search_sublemmas=True))
display(generate_wordforms_with_query_extender('nato', search_sublemmas=True))

In [ ]:
display(generate_wordforms_with_query_extender('Euroopa', search_sublemmas=True))
display(generate_wordforms_with_query_extender('euroopa', search_sublemmas=True))

# Veebiteenuse poolt väljakutsutav veebiteenus

<u>Universiaalsed veebiteenustel põhinev lahendus</u>

* [analyser](https://github.com/estnltk/smart-search/tree/main/api/api_analyser) -- tekstide morfoloogiline analüsaator 
* [generator](https://github.com/estnltk/smart-search/tree/main/api/api_generator) -- sõnavormide generaator
* [speller](https://github.com/estnltk/smart-search/tree/main/api/api_speller) -- kirjavigade parandaja

Lahenduse põhimõtteline kondikava
```
wordform = spell_text(search_word)
lemma = analyze_text(wordform)
search_forms = generate_all_worldforms(lemma)
sql_statement = f'SELECT * FROM table WHERE {sql_or(search_forms)}'
...
```
Universaalne lahendus, mis lükkab optimeeringu andmebaasimootori kaela.


In [ ]:
SQL_list = '[{}]'.format(
    ', '.join("'{}'".format(word) for word in generate_all_wordforms('president')['wordform']))
SQL_query = f'SELECT * FROM captions\nWHERE word in {SQL_list};'
print(SQL_query)

# Veebiteenuse poolt väljakutsutav veebiteenus


<u>Riigi Teataja tekstide sisu arvestav lahendus</u>

* [query_extender](https://github.com/estnltk/smart-search/tree/main/api/api_query_extender) -- laiendab otsisõna minimaalseks hulgaks RT veebiteenusele suunatud sisendiks. 

Lahenduse põhimõtteline kondikava

```
search_forms = extend_query(search_word)
sql_statement = f'SELECT * FROM table WHERE {sql_or_criterion(search_forms)}'
...
```

**Demo rakendus:** [rt_pealkirjaotsing](https://smart-search.tartunlp.ai/rt_pealkirjaotsing/process)

# Andmebaasi sisu muutev veebiteenus

<u>Universaalne lahendus tekstide indekseerimiseks </u>
* [advanced_indexing](https://github.com/estnltk/smart-search/tree/main/api/api_advanced_indexing)

Lahenduse põhimõtteline kondikava

```
for document in added_documents:
    index = advanced_indexing(document)
    index_text = format_for_db_search(index) 
    insert_into_db(index_text)
```

Selle tulemusena tekib andmebaasisüsteemi paremini otsitavad tekstide indeksid ning kõik tekstivormid saab kätte otsides ainult sõnade algvorme.





In [ ]:
text = 'Presidendi ametiraha seadus'
output_tables = ['lemma_kõik_vormid', 'lemma_korpuse_vormid', 'liitsõnad']
        
ANALYZER_QUERY =  "https://smart-search.tartunlp.ai/api/advanced_indexing/json" 
HEADERS = {"Content-Type": "application/json"}
POST_DATA_TEMPLATE = {"params":{"tables": output_tables}, "sources": {'doc_42': {"content": text}}}
    
response = requests.post(ANALYZER_QUERY, json=POST_DATA_TEMPLATE, headers=HEADERS)
assert response.ok, "Webservice failed"
response = response.json()
print(list(response.keys()))
print(list(response['tabelid'].keys()))

In [ ]:
print(json.dumps(response['tabelid']['liitsõnad'], indent=4))

In [ ]:
from predefined_functions import index_text
index_text(doc_id='doc_1', text='Presidendi ametiraha seadus', output_tables='indeks_lemmad') #liitsõnad
#index_text(doc_id='doc_1', text='Presidendi ametiraha seadus', output_tables='lemma_korpuse_vormid')

# Päringulaiendaja kohandamine

[**Bashi skript**](https://github.com/estnltk/smart-search/tree/main/scripts/query_extender_setup/example_script_based_workflow)
* Konfitakse läbi keskonnamuutujate.
* Eeldab spetsiifilist kataloogistruktuuri.
* Loob ja uuendab veebiteenuse `query_extender` sisemist andmebaasi. 

[**Näitetöövoog**](https://github.com/estnltk/smart-search/tree/main/demod/toovood/riigi_teataja_pealkirjaotsing)
* Kollektsioon Jupyteri tööraamatuid, mis demonstreerivad analüüsi kõiki samme.
* Võimaldab avastada skriptide poolt käsitlemata ootamatusi.